In [2]:
!pip install scipy
!pip install pysindy plotly


In [3]:
import numpy as np
import plotly.graph_objs as go
from scipy.integrate import solve_ivp

# Parameters for the Lorenz system
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0

# Lorenz system equations
def lorenz(t, state):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Initial conditions and integration time
initial_state = [1.0, 1.0, 1.0]
t_span = (0, 49)  # Time interval
t_eval = np.linspace(t_span[0], t_span[1], 10000)  # Time points

# Solve the Lorenz system
sol = solve_ivp(lorenz, t_span, initial_state, t_eval=t_eval)

# Extract data
x = sol.y[0]  # Time series x(t)

# Choose delay and embedding dimension
tau = 13  # Delay in time steps (chosen number, can be experimented with)
embedding_dim = 3  # Embedding dimension

# Function to create delay vectors
# def delay_embedding(data, delay, dim):
#     n = len(data)
#     embedded_data = np.zeros((n - (dim - 1) * delay, dim))
#     for i in range(dim):
#         embedded_data[:, i] = data[i * delay:n - (dim - 1 - i) * delay]
#     return embedded_data

def delay_embedding(data, delay, dim):
    n = len(data)
    embedded_data = np.zeros((n - (dim - 1) * delay, dim))
    embedded_data[:, 0] = data[0:n - (dim - 1) * delay]
    embedded_data[:, 1] = data[delay:n - (dim - 1 - 1) * delay]
    embedded_data[:, 2] = data[2 * delay:n - (dim - 1 - 2) * delay]
    return embedded_data

# Create delay vectors
embedded_data = delay_embedding(x, tau, embedding_dim + 1)

# Create an interactive Plotly graph
fig = go.Figure(data=[go.Scatter3d(
    x=embedded_data[:, 0],
    y=embedded_data[:, 1],
    z=embedded_data[:, 2],
    mode='lines',
    line=dict(color='blue', width=2)
)])

# Configure the layout of the graph
fig.update_layout(
    title="Reconstructed Lorenz Attractor using Delay Embedding",
    scene=dict(
        xaxis_title='x(t)',
        yaxis_title=f'x(t-{tau})',
        zaxis_title=f'x(t-{2*tau})'
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, b=0, t=40)
)

# Display the graph
fig.show()


In [4]:
# Create an interactive Plotly graph
fig = go.Figure(data=[go.Scatter3d(
    x=sol.y[0],
    y=sol.y[1],
    z=sol.y[2],
    mode='lines',
    line=dict(color='blue', width=2)
)])

# Configure the layout of the graph
fig.update_layout(
    title="Reconstructed Lorenz Attractor using Delay Embedding",
    scene=dict(
        xaxis_title='x(t)',
        yaxis_title=f'x(t-{tau})',
        zaxis_title=f'x(t-{2*tau})'
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, b=0, t=40)
)

# Display the graph
fig.show()

In [5]:
import numpy as np
import pysindy as ps
from scipy.integrate import solve_ivp
import plotly.graph_objs as go

# Параметры системы Лоренца
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0

# Уравнения Лоренца
def lorenz(t, state):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Начальные условия и время интегрирования
initial_state = [1.0, 1.0, 1.0]
t_span = (0, 25)  # временной интервал
t_eval = np.linspace(t_span[0], t_span[1], 1000)  # временные точки

# Интеграция системы
sol = solve_ivp(lorenz, t_span, initial_state, t_eval=t_eval)
x = sol.y[0]
y = sol.y[1]
z = sol.y[2]

# Стек данных (X)
data = np.vstack((x, y, z)).T

# Задержка времени (шаг)
dt = t_eval[1] - t_eval[0]

# Обучение модели SINDy
model = ps.SINDy()
model.fit(data, t=dt)
model.print()

# Проверка модели на тестовом временном ряде
x_test = x + 0.1 * np.random.randn(*x.shape)  # Добавление шума
y_test = y + 0.1 * np.random.randn(*y.shape)
z_test = z + 0.1 * np.random.randn(*z.shape)
data_test = np.vstack((x_test, y_test, z_test)).T

# Прогнозирование и визуализация
simulated_data = model.simulate(initial_state, t_eval)

# Построение интерактивного графика с Plotly
fig = go.Figure()

# Исходные данные
fig.add_trace(go.Scatter3d(
    x=data[:, 0], y=data[:, 1], z=data[:, 2],
    mode='lines', name='Original',
    line=dict(color='blue', width=2)
))

# Симулированные данные
fig.add_trace(go.Scatter3d(
    x=simulated_data[:, 0], y=simulated_data[:, 1], z=simulated_data[:, 2],
    mode='lines', name='Simulated',
    line=dict(color='red', width=2, dash='dash')
))

# Настройки графика
fig.update_layout(
    title='Lorenz System Reconstruction using SINDy',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    width=800,
    height=600
)

# Отображение графика
fig.show()


(x0)' = -9.864 x0 + 9.864 x1
(x1)' = -0.213 1 + 26.540 x0 + -0.712 x1 + -0.958 x0 x2
(x2)' = 0.122 1 + -2.621 x2 + 0.980 x0 x1
